# Question Generator Notebook

This notebook is dedicated to generating test questions for chatbot testing.
It creates realistic customer service scenarios and exports them to JSON files.

## Features:
- Generate diverse customer service questions
- Customize scenarios and customer types
- Export questions with metadata (customer type, complexity)
- Save to timestamped files for use in chatbot testing

## Usage:
1. Run cells in order from top to bottom
2. Customize the generation prompt and model
3. Generate questions
4. Review and edit questions if needed
5. Export to JSON file for use in chatbot_tester.ipynb

In [1]:
# Import required libraries
import json
import yaml
import os
import sys
from datetime import datetime
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Set the working directory to the root of the project
os.chdir('/workspace')

# Add workspace to path for imports
sys.path.insert(0, '/workspace')

# Import our system components
from src.integrations.llm_providers import LLMProviderFactory

print("✅ All libraries imported successfully!")
print("Ready to generate test questions.")

✅ All libraries imported successfully!
Ready to generate test questions.


## Step 1: Load Available Models

Load the available models for question generation.

In [2]:
# Load available models
config_base_path = Path('/workspace/config')

# Load shared models for generation options
with open(config_base_path / 'shared' / 'models.yaml', 'r') as f:
    shared_models = yaml.safe_load(f)

available_models = list(shared_models['model_aliases'].keys())
print(f"📋 Available models: {available_models}")
print(f"✅ Configuration loaded successfully!")

📋 Available models: ['anthropic_general_budget', 'anthropic_general_standard', 'anthropic_reasoning_premium', 'anthropic_coding_premium', 'anthropic_flagship', 'openai_general_standard', 'openai_general_budget', 'openai_coding_standard', 'local_general_standard', 'local_general_premium', 'local_coding_standard', 'local_general_budget']
✅ Configuration loaded successfully!


## Step 2: Configure Question Generation

Customize the settings for generating test questions.

In [3]:
# Configuration widgets for question generation
print("🔧 Question Generation Configuration")

# Number of questions to generate
question_count = widgets.IntSlider(
    value=20,
    min=5,
    max=50,
    step=5,
    description='Number of questions:',
    style={'description_width': 'initial'}
)

# Model selection
generation_model = widgets.Dropdown(
    options=available_models,
    value='anthropic_general_standard',
    description='Generation model:',
    style={'description_width': 'initial'}
)

# Scenario prompt
scenario_prompt = widgets.Textarea(
    value="""Generate chatbot questions from users contacting their insurance company about their coverage. 
Include these customer types:
- Frustrated customers (angry about rates, claim denials, poor service)
- Confused customers (need simple explanations, don't understand terms)
- Urgent customers (immediate needs like accidents, claims, proof of insurance)
- Normal customers (routine inquiries, policy changes, general questions)

Include these complexity levels:
- Simple: Basic questions that can be answered directly
- Medium: Questions requiring some explanation or context
- Complex: Multi-part questions or situations requiring detailed responses

Make the questions realistic and diverse.""",
    placeholder="Describe the scenario for question generation...",
    description="Scenario description:",
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Display configuration widgets
display(question_count)
display(generation_model)
print("\nScenario for question generation:")
display(scenario_prompt)

🔧 Question Generation Configuration


IntSlider(value=20, description='Number of questions:', max=50, min=5, step=5, style=SliderStyle(description_w…

Dropdown(description='Generation model:', index=1, options=('anthropic_general_budget', 'anthropic_general_sta…


Scenario for question generation:


Textarea(value="Generate chatbot questions from users contacting their insurance company about their coverage.…

## Step 3: Generate Questions

Generate test questions based on the configuration above.

In [4]:
def generate_test_questions(prompt, model_name, count=20):
    """Generate test questions using the specified model"""
    print(f"🤖 Generating {count} test questions using {model_name}...")
    
    # Create a generation prompt
    generation_system_prompt = f"""
    Generate exactly {count} realistic customer service questions based on this scenario:
    {prompt}
    
    Return the questions as a JSON array with this format:
    [
        {{"id": 1, "question": "Question text here", "customer_type": "frustrated/confused/urgent/normal", "complexity": "simple/medium/complex"}},
        {{"id": 2, "question": "Another question", "customer_type": "normal", "complexity": "medium"}}
    ]
    
    Guidelines:
    - Make questions diverse and realistic
    - Include variety in customer emotional states
    - Mix complexity levels appropriately
    - Use natural language that real customers would use
    - Include specific details that make questions authentic
    
    IMPORTANT: Return ONLY the JSON array, no other text.
    """
    
    try:
        # Initialize model provider factory
        factory = LLMProviderFactory()
        provider = factory.create_provider(model_name)
        
        # Generate questions
        response = provider.generate_response(
            prompt="Generate the test questions now.",
            system_prompt=generation_system_prompt
        )
        
        print(f"📝 Generated response (first 200 chars): {response[:200]}...")
        
        # Clean up the response - remove any markdown formatting
        cleaned_response = response.strip()
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:]  # Remove ```json
        if cleaned_response.endswith('```'):
            cleaned_response = cleaned_response[:-3]  # Remove ```
        cleaned_response = cleaned_response.strip()
        
        # Parse JSON response
        questions_data = json.loads(cleaned_response)
        
        # Ensure we have a list
        if isinstance(questions_data, dict):
            questions_data = [questions_data]
        elif not isinstance(questions_data, list):
            raise ValueError(f"Expected list or dict, got {type(questions_data)}")
        
        # Validate each question has required fields
        validated_questions = []
        for i, q in enumerate(questions_data):
            if isinstance(q, dict):
                # Ensure required fields exist
                validated_q = {
                    "id": q.get("id", i + 1),
                    "question": q.get("question", f"Question {i + 1}"),
                    "customer_type": q.get("customer_type", "normal"),
                    "complexity": q.get("complexity", "medium")
                }
                validated_questions.append(validated_q)
            else:
                # Convert string to dict if needed
                validated_q = {
                    "id": i + 1,
                    "question": str(q),
                    "customer_type": "normal",
                    "complexity": "medium"
                }
                validated_questions.append(validated_q)
        
        return validated_questions
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print(f"📝 Raw response that failed to parse: {response}")
        # Return sample questions as fallback
        return create_sample_questions(count)
    except Exception as e:
        print(f"❌ Error generating questions: {e}")
        # Return sample questions as fallback
        return create_sample_questions(count)

def create_sample_questions(count=5):
    """Create sample questions as fallback"""
    sample_questions = [
        {"id": 1, "question": "I need to file a claim for my car accident yesterday", "customer_type": "urgent", "complexity": "medium"},
        {"id": 2, "question": "What does my insurance cover exactly?", "customer_type": "confused", "complexity": "simple"},
        {"id": 3, "question": "Why did my premium go up this month? I haven't had any accidents!", "customer_type": "frustrated", "complexity": "medium"},
        {"id": 4, "question": "Can I add a new driver to my policy?", "customer_type": "normal", "complexity": "simple"},
        {"id": 5, "question": "My house was damaged in a storm and I need to understand my coverage options for temporary housing", "customer_type": "urgent", "complexity": "complex"}
    ]
    return sample_questions[:count]

# Generate questions using current configuration
test_questions = generate_test_questions(
    scenario_prompt.value,
    generation_model.value,
    question_count.value
)

print(f"\n✅ Generated {len(test_questions)} test questions!")

# Display preview
if test_questions:
    print("\n📋 Preview of generated questions:")
    for i, q in enumerate(test_questions[:3]):
        print(f"  {i+1}. {q['question']} [{q['customer_type']}]")
    if len(test_questions) > 3:
        print(f"  ... and {len(test_questions) - 3} more questions")

🤖 Generating 20 test questions using anthropic_general_standard...
✅ Creating LLM provider: anthropic_general_standard → claude-3-5-sonnet-20241022 (anthropic)
✅ 17:43:45.709 [INFO    ] claude-3-5-sonnet-20241022 | Initializing LLM provider | model_name=claude-3-5-sonnet-20241022
✅ 17:43:45.715 [INFO    ] claude-3-5-sonnet-20241022 | LLM provider initialized successfully | model_name=claude-3-5-sonnet-20241022
✅ 17:43:59.832 [INFO    ] claude-3-5-sonnet-20241022 | Model call: claude-3-5-sonnet-20241022 - generate_response | model_name=claude-3-5-sonnet-20241022 operation=generate_response
📝 Generated response (first 200 chars): [
    {"id": 1, "question": "Why did my premium increase by $200? This is ridiculous - I haven't had any claims!", "customer_type": "frustrated", "complexity": "medium"},
    {"id": 2, "question": "I ...

✅ Generated 20 test questions!

📋 Preview of generated questions:
  1. Why did my premium increase by $200? This is ridiculous - I haven't had any claims! [fru

## Step 4: Review and Edit Questions

Review the generated questions and make edits if needed.

In [5]:
# Display questions for review
if test_questions:
    print("📝 Generated Questions Review")
    print("Review the questions below and edit the JSON if needed.\n")
    
    # Convert to DataFrame for easy viewing
    df = pd.DataFrame(test_questions)
    display(df)
    
    # Show distribution
    print("\n📊 Distribution Analysis:")
    print("\nBy Customer Type:")
    print(df['customer_type'].value_counts())
    print("\nBy Complexity:")
    print(df['complexity'].value_counts())
    
    # JSON editor for manual edits
    questions_json = widgets.Textarea(
        value=json.dumps(test_questions, indent=2),
        description="Questions JSON:",
        layout=widgets.Layout(width='100%', height='300px'),
        style={'description_width': 'initial'}
    )
    
    print("\n🔧 Edit questions as JSON (optional):")
    display(questions_json)
    
    def update_questions_from_json():
        """Update questions from the JSON editor"""
        global test_questions
        try:
            updated_questions = json.loads(questions_json.value)
            test_questions = updated_questions
            print("✅ Questions updated from JSON editor")
            
            # Show updated distribution
            df_updated = pd.DataFrame(test_questions)
            print("\n📊 Updated Distribution:")
            print("\nBy Customer Type:")
            print(df_updated['customer_type'].value_counts())
            print("\nBy Complexity:")
            print(df_updated['complexity'].value_counts())
            
        except Exception as e:
            print(f"❌ Error parsing JSON: {e}")
    
    # Button to update from JSON
    update_btn = widgets.Button(
        description="Update from JSON",
        button_style='primary'
    )
    update_btn.on_click(lambda b: update_questions_from_json())
    display(update_btn)
    
else:
    print("⚠️ No questions generated. Please run the previous cell first.")

📝 Generated Questions Review
Review the questions below and edit the JSON if needed.



,id,question,customer_type,complexity
0,1,Why did my premium increase by $200? This is r...,frustrated,medium
1,2,I just had a car accident. What do I need to d...,urgent,simple
2,3,Can someone explain what a deductible is? I ke...,confused,simple
3,4,How do I add my teenage daughter to my auto po...,normal,simple
4,5,I've been a customer for 15 years and you deni...,frustrated,complex
5,6,What's the difference between comprehensive an...,confused,medium
6,7,My house just flooded - I need emergency assis...,urgent,complex
7,8,Can I get proof of insurance sent to my phone?...,urgent,simple
8,9,How do I update my billing information?,normal,simple
9,10,What exactly is an 'act of God' and why isn't ...,confused,medium



📊 Distribution Analysis:

By Customer Type:
customer_type
frustrated    5
urgent        5
confused      5
normal        5
Name: count, dtype: int64

By Complexity:
complexity
medium     7
simple     7
complex    6
Name: count, dtype: int64

🔧 Edit questions as JSON (optional):


Textarea(value='[\n  {\n    "id": 1,\n    "question": "Why did my premium increase by $200? This is ridiculous…

Button(button_style='primary', description='Update from JSON', style=ButtonStyle())

## Step 5: Export Questions

Export the generated questions to a timestamped JSON file for use in chatbot testing.

In [6]:
# Export questions to file
if test_questions:
    # Create timestamp for filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    # Create test_questions directory if it doesn't exist
    questions_dir = Path('/workspace/notebooks/test_questions')
    questions_dir.mkdir(exist_ok=True)
    
    # Export questions with metadata
    questions_filename = f"test_questions_{timestamp}.json"
    questions_path = questions_dir / questions_filename
    
    # Create export data with metadata
    export_data = {
        "metadata": {
            "generation_timestamp": timestamp,
            "generation_model": generation_model.value,
            "question_count": len(test_questions),
            "scenario_prompt": scenario_prompt.value,
            "distribution": {
                "customer_types": pd.DataFrame(test_questions)['customer_type'].value_counts().to_dict(),
                "complexities": pd.DataFrame(test_questions)['complexity'].value_counts().to_dict()
            }
        },
        "questions": test_questions
    }
    
    # Save to file
    with open(questions_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"✅ Questions exported successfully!")
    print(f"📄 File: {questions_path}")
    print(f"🔢 Total questions: {len(test_questions)}")
    
    # Show export summary
    df_export = pd.DataFrame(test_questions)
    print(f"\n📊 Export Summary:")
    print(f"  Timestamp: {timestamp}")
    print(f"  Generation model: {generation_model.value}")
    print(f"  Customer types: {dict(df_export['customer_type'].value_counts())}")
    print(f"  Complexities: {dict(df_export['complexity'].value_counts())}")
    
    # Show usage instructions
    print(f"\n💡 Usage Instructions:")
    print(f"  1. Use this file in chatbot_tester.ipynb")
    print(f"  2. Select 'Load from file' option")
    print(f"  3. Upload: {questions_filename}")
    print(f"  4. The chatbot tester will load these questions automatically")
    
    # Create simple questions-only file for easier loading
    simple_filename = f"questions_only_{timestamp}.json"
    simple_path = questions_dir / simple_filename
    
    with open(simple_path, 'w') as f:
        json.dump(test_questions, f, indent=2)
    
    print(f"\n📄 Also created simple format: {simple_filename}")
    print(f"   (Contains only questions array for direct loading)")
    
else:
    print("⚠️ No questions to export. Please generate questions first.")

✅ Questions exported successfully!
📄 File: /workspace/notebooks/test_questions/test_questions_20250718_1744.json
🔢 Total questions: 20

📊 Export Summary:
  Timestamp: 20250718_1744
  Generation model: anthropic_general_standard
  Customer types: {'frustrated': np.int64(5), 'urgent': np.int64(5), 'confused': np.int64(5), 'normal': np.int64(5)}
  Complexities: {'medium': np.int64(7), 'simple': np.int64(7), 'complex': np.int64(6)}

💡 Usage Instructions:
  1. Use this file in chatbot_tester.ipynb
  2. Select 'Load from file' option
  3. Upload: test_questions_20250718_1744.json
  4. The chatbot tester will load these questions automatically

📄 Also created simple format: questions_only_20250718_1744.json
   (Contains only questions array for direct loading)


## Next Steps

🎉 **Questions Generated Successfully!**

### What you've created:
- ✅ Generated realistic customer service test questions
- ✅ Included diverse customer types (frustrated, confused, urgent, normal)
- ✅ Varied complexity levels (simple, medium, complex)
- ✅ Exported to timestamped JSON files with metadata

### Your exported files:
- **Full export**: Contains questions + generation metadata
- **Simple export**: Contains only questions array for direct loading

### How to use with chatbot_tester.ipynb:
1. Open `chatbot_tester.ipynb`
2. In the input section, select "Load from file"
3. Upload your generated questions file
4. The chatbot will test against your custom questions

### Generate more questions:
- Modify the scenario prompt above for different contexts
- Try different models for varied question styles
- Adjust question count for different test sizes
- Edit the JSON manually for fine-tuning

---
*This notebook is part of the Human-in-the-Loop AI System for customer service testing.*